In [3]:
from selenium import webdriver
from time import sleep
import requests
import pymysql
import bs4

url = 'https://www.worldometers.info/'
browser = webdriver.Firefox()
browser.get(url)
sleep(2)

soup = bs4.BeautifulSoup(browser.page_source, 'html.parser')
browser.close()

In [12]:
counters = [{'c1': 'World Pop'}, {'c31': 'Obese People'}, {'c30': 'Overweight People'}, {'c29': 'Undernourished'}]
data = {}

for counter in counters:
    html = soup.find('div', {'id' : counter}).findChildren('span', {'class' : 'rts-counter'})
    data[list(counter.keys())[0]] = {'counter_id' : list(counter.keys())[0], 'counter_name' : list(counter.values())[0], 'count' : int("".join([i.text if i.text != ',' else '' for i in html[0].findChildren()]))}

print(data)

{'c1': {'counter_id': 'c1', 'counter_name': 'World Pop', 'count': 7773683854}, 'c31': {'counter_id': 'c31', 'counter_name': 'Obese People', 'count': 753162663}, 'c30': {'counter_id': 'c30', 'counter_name': 'Overweight People', 'count': 1690964715}, 'c29': {'counter_id': 'c29', 'counter_name': 'Undernourished', 'count': 841840550}}


In [15]:
cnx = pymysql.connect(user='dev', password='ax2',host='127.0.0.1',port=3306,db='test')

query = "INSERT INTO worldpop (counterId, counterName, count) VALUES (%(counter_id)s, %(counter_name)s, %(count)s)"

with cnx.cursor() as cursor:
    for counter in data:
        cursor.execute(query, data[counter])

cnx.commit()
cnx.close()